In [1]:
from poisson_fem import PoissonFEM
import numpy as np
import scipy.sparse as sps
import scipy.sparse.linalg as lg
import time
import petsc4py
import sys
petsc4py.init(sys.argv)
from petsc4py import PETSc

In [2]:
m = 8
mesh = PoissonFEM.RectangularMesh(np.ones(m)/m)
# mesh.plot()

In [3]:
def origin(x):
    return np.abs(x[0]) < np.finfo(float).eps and np.abs(x[1]) < np.finfo(float).eps
def essBoundaryFun(x):
    return 1.0
mesh.setEssentialBoundary(origin, essBoundaryFun)

In [4]:
def domainBoundary(x):
    # unit square
    return np.abs(x[0]) < np.finfo(float).eps or np.abs(x[1]) < np.finfo(float).eps or \
            np.abs(x[0]) > 1.0 - np.finfo(float).eps or np.abs(x[1]) > 1.0 - np.finfo(float).eps
mesh.setNaturalBoundary(domainBoundary)

In [5]:
#Define boundary flux field
def flux(x):
    a = np.array([1, 2, 3])
    q = np.array([a[0] + a[2]*x[1], a[1] + a[2]*x[0]])
    return q

In [6]:
rhs = PoissonFEM.RightHandSide(mesh)

In [7]:
rhs.setNaturalRHS(mesh, flux)

In [8]:
funSpace = PoissonFEM.FunctionSpace(mesh)

In [9]:
K = PoissonFEM.StiffnessMatrix(mesh, funSpace)

In [10]:
rhs.setRhsStencil(mesh, K)

In [11]:
lmbda_np = np.arange(5.0, m**2 + 5.0)
lmbda = PETSc.Vec().createSeq(mesh.nCells)
rangeCells = range(mesh.nCells)
lmbda.setValues(rangeCells, lmbda_np)

In [12]:
# Precomputations
x = PETSc.Vec().createSeq(mesh.nEq)

In [13]:
# Set up solver
ksp = PETSc.KSP().create()
ksp.setType('preonly')
pc = ksp.getPC()
pc.setType('cholesky')
ksp.setFromOptions() #???

In [14]:
N = 1e3

In [15]:
start = time.time()
for n in range(int(N)):
    # Stiffness matrix assembly
    K.assemble(lmbda)
    # RHS assembly
    rhs.assemble(lmbda)
    # solving
    ksp.setOperators(K.matrix)
    ksp.solve(rhs.vector, x)
petsc_time = (time.time() - start)/N
print('PETSc time = ', petsc_time)

PETSc time =  3.771090507507324e-05


In [16]:
# for scipy comparison
Kes = K.globStiffStencil.getValuesCSR()
Kes = sps.csr_matrix((Kes[2], Kes[1], Kes[0]))

In [17]:
start = time.time()
for n in range(int(N)):
    Kvecs = Kes @ lmbda.array
    K1 = sps.csr_matrix((Kvecs[K.vec_nonzero], K.indices, K.indptr))
    rhs.assemble(lmbda)   # for fair comparison
    x1 = lg.spsolve(K1, rhs.vector.array)
scipy_time = (time.time() - start)/N
print('scipy time = ', scipy_time)

scipy time =  0.0002895774841308594


In [18]:
diff = np.linalg.norm(x.array - x1)/np.linalg.norm(x1)
print('result difference = ', diff)
print('PETSc speedup = ', scipy_time/petsc_time)

result difference =  1.5062426929228647e-13
PETSc speedup =  7.678879187714562


In [19]:
print(x.array)

[1.02278352 1.03570934 1.04495548 1.05272428 1.06002749 1.0675361
 1.07593055 1.08577733 1.06292587 1.07902031 1.08924891 1.09710833
 1.10395058 1.11052592 1.11736786 1.1249899  1.13438647 1.09232965
 1.10428562 1.11421532 1.12259286 1.13015478 1.13740198 1.14472141
 1.15248623 1.16097157 1.11139252 1.12170986 1.13110033 1.13969864
 1.14774799 1.15550532 1.16319036 1.17098383 1.17901013 1.12553694
 1.13495295 1.14399687 1.15266754 1.16101623 1.16911716 1.17704457
 1.1848592  1.19259955 1.13660956 1.14548309 1.15433617 1.16309124
 1.17168348 1.18006588 1.18820268 1.1960629  1.20362235 1.14543701
 1.15398358 1.16280162 1.17173273 1.18061139 1.18930312 1.19769074
 1.20565907 1.21311848 1.15238074 1.16076725 1.16976667 1.17902634
 1.18829713 1.19738767 1.20612792 1.21433627 1.22177511 1.15728896
 1.1659988  1.17547811 1.18525934 1.19506086 1.20467241 1.21390662
 1.2225552  1.23031413]


In [20]:
print(x1)

[1.02278352 1.03570934 1.04495548 1.05272428 1.06002749 1.0675361
 1.07593055 1.08577733 1.06292587 1.07902031 1.08924891 1.09710833
 1.10395058 1.11052592 1.11736786 1.1249899  1.13438647 1.09232965
 1.10428562 1.11421532 1.12259286 1.13015478 1.13740198 1.14472141
 1.15248623 1.16097157 1.11139252 1.12170986 1.13110033 1.13969864
 1.14774799 1.15550532 1.16319036 1.17098383 1.17901013 1.12553694
 1.13495295 1.14399687 1.15266754 1.16101623 1.16911716 1.17704457
 1.1848592  1.19259955 1.13660956 1.14548309 1.15433617 1.16309124
 1.17168348 1.18006588 1.18820268 1.1960629  1.20362235 1.14543701
 1.15398358 1.16280162 1.17173273 1.18061139 1.18930312 1.19769074
 1.20565907 1.21311848 1.15238074 1.16076725 1.16976667 1.17902634
 1.18829713 1.19738767 1.20612792 1.21433627 1.22177511 1.15728896
 1.1659988  1.17547811 1.18525934 1.19506086 1.20467241 1.21390662
 1.2225552  1.23031413]
